# Semantic SfM of single scene in Scannet

### 1a. Download entire Scannet dataset

Find information on ScanNet website: http://www.scan-net.org/ScanNet/

After download the data, I should extract data from .sens using [SensReader/reader.py](https://github.com/ScanNet/ScanNet/tree/master/SensReader/python). To extract all scans in the test data, I wrote a script to automate executing the python script, `reader.py`:

```python
import os
import subprocess
import numpy as np

# Define the base directory for the scans_test
base_dir = "/home/zchen256/semantic_SfM/data/scannet/scans"

# Iterate over all files in the directory
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".sens"):
            # 5% chance of extracting .sens files
            if np.random.rand() > 0.05:
                continue
            # Construct the full path to the .sens file
            sens_file = os.path.join(root, file)
            # Construct the output path
            output_path = os.path.join(root, "output")
            # Create the output directory if it doesn't exist
            os.makedirs(output_path, exist_ok=True)
            # Construct the command
            command = [
                "python3", "reader.py",
                "--filename", sens_file,
                "--output_path", output_path,
                "--export_depth_images",
                "--export_color_images",
                "--export_poses",
                "--export_intrinsics"
            ]
            # Run the command
            print(command)
            subprocess.run(command)
```

### 1b. Extract Scannet data to prepare SSfM data in batch

In [1]:
from scene_extractor import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def batch_extract_scenes(scene_dir, save_dir, training=False):
    # iterate over folders in scene_dir
    for scene_folder in os.listdir(scene_dir):
        scene_folder_path = os.path.join(scene_dir, scene_folder)
        save_folder_path = os.path.join(save_dir, scene_folder)
        output_folder_path = os.path.join(scene_folder_path, 'output')
        if not os.path.exists(output_folder_path):
            print(f'Error: {scene_folder_path} does not exist')
            continue
        if not os.path.exists(save_folder_path):
            os.makedirs(save_folder_path)
        try:
            scene_extractor = SceneExtractor(scene_folder_path, save_folder_path)
            print(f'Extracting scene {scene_folder}')
            if training:
                scene_extractor.extract_segmentations()
                scene_extractor.extract_ground_truth()
        except Exception as e:
            print(f'Error in extracting scene {scene_folder}: {e}')
            continue

In [4]:
scan_dir = '../../data/scannet/scans'
save_dir = '../../data/scannet/ssfm_train'

batch_extract_scenes(scan_dir, save_dir, training=True)

Error in extracting scene scene0586_01: Extract scene folder does not exist!
Error in extracting scene scene0705_00: Extract scene folder does not exist!
Error in extracting scene scene0667_00: Extract scene folder does not exist!
Error in extracting scene scene0620_01: Extract scene folder does not exist!
Error in extracting scene scene0638_00: Extract scene folder does not exist!
Error in extracting scene scene0454_00: Extract scene folder does not exist!
Error in extracting scene scene0435_03: Extract scene folder does not exist!
Error in extracting scene scene0536_00: Extract scene folder does not exist!
Error in extracting scene scene0472_02: Extract scene folder does not exist!
Error in extracting scene scene0571_01: Extract scene folder does not exist!
Error in extracting scene scene0690_00: Extract scene folder does not exist!
Error in extracting scene scene0569_00: Extract scene folder does not exist!
Error in extracting scene scene0510_02: Extract scene folder does not exist!

In [15]:
import os
import yaml

scene_dir = '../../data/scannet/ssfm_valid/scene0011_00'

# batch project
photos_folder_path = os.path.join(scene_dir, 'photos')
associations_folder_path = os.path.join(scene_dir, 'associations')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
associations_folder_path = os.path.join(scene_dir, 'associations')

### 2. Select keyimages based on blur

In [3]:
from select_keyimages import select_scannet_keyimages

In [45]:
select_scannet_keyimages(scene_dir, ratio=0.2, threshold=180, file_cluster_size=30, file_select_window=10, n_jobs=8)

Processing images: 100%|██████████| 2374/2374 [00:10<00:00, 222.30it/s]


Number of keyimages_threshold:  537
Number of keyimages_ratio:  474
Number of keyimages:  537
Number of selected keyimages from window:  238
Number of selected keyimages:  255
Total images:  2374
Keyimages saved to:  ../../data/scannet/ssfm_valid/scene0011_00/associations/keyimages.yaml


### 3a. Segment keyimages

In [4]:
from ssfm.image_segmentation import ImageSegmentation

In [56]:
sam_params = {}
sam_params['model_name'] = 'sam'
sam_params['model_path'] = '../../semantic_SfM/sam/sam_vit_h_4b8939.pth'
sam_params['model_type'] = 'vit_h'
sam_params['device'] = 'cuda:1'
sam_params['points_per_side'] = 32
sam_params['pred_iou_thresh'] = 0.96
sam_params['stability_score_thresh'] = 0.96
sam_params['crop_n_layers'] = 1

In [16]:
using_keyimages = True

if using_keyimages:
    keyimages_path = os.path.join(scene_dir, 'associations', 'keyimages.yaml')
    # read keyimages
    with open(keyimages_path, 'r') as f:
        keyimages = yaml.load(f, Loader=yaml.FullLoader)

    # replace .npy with .jpg
    images = [keyimage.replace('.npy', '.jpg') for keyimage in keyimages]
    # sort images based on the values of keyimages in file names
    images = sorted(images, key=lambda x: int(x.split('_')[-1].split('.')[0]))
    image_paths = [os.path.join(scene_dir, 'photos', image) for image in images]

else:
    image_paths = [os.path.join(scene_dir, 'photos', image) for image in os.listdir(os.path.join(scene_dir, 'photos'))]
    # sort images based on the values of keyimages in file names
    image_paths = sorted(image_paths, key=lambda x: int(x.split('_')[-1].split('.')[0]))

print(image_paths)
print(len(image_paths))

['../../data/scannet/ssfm_valid/scene0011_00/photos/3.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/14.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/29.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/32.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/42.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/50.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/63.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/78.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/83.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/99.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/106.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/117.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/124.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/129.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/134.jpg', '../../data/scannet/ssfm_valid/scene0011_00/photos/146.jpg', '../../data/scannet/ssfm_valid/sce

In [ ]:
image_segmentor = ImageSegmentation(sam_params)   
#image_segmentor.set_distortion_correction('../data/courtright/SfM_products/agisoft_cameras.xml')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations')
image_segmentor.batch_predict(image_paths, segmentations_folder_path, maximum_size=10000, save_overlap=True)

### 4. Create projection associations

In [18]:
from ssfm.probabilistic_projection import *

In [19]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.005)
pointcloud_projector.read_scannet_camera_parameters(scene_dir)
mesh_file_path = os.path.join(scene_dir, 'reconstructions', 'mesh_vertices_color.npy')
pointcloud_projector.read_scannet_mesh(mesh_file_path)

In [20]:
assert os.path.exists(keyimages_path), 'Keyimages not found'

with open(keyimages_path, 'r') as f:
    keyimages = yaml.safe_load(f)

# replace .npy with .jpg in keyimages, keyimages is a list of strings
image_list = [os.path.splitext(image)[0] + '.jpg' for image in keyimages]
image_list = sorted(image_list, key=lambda x: int(x.split('_')[-1].split('.')[0]))

In [21]:
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, associations_folder_path, num_workers=16, save_depth=False)

Processing frames: 100%|██████████| 255/255 [00:26<00:00,  9.62it/s]


In [22]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [23]:
print(len(image_list))
smc_solver = KeyimageAssociationsBuilder(image_list, associations_folder_path, segmentations_folder_path)
smc_solver.build_associations()

255


100%|██████████| 255/255 [00:01<00:00, 190.54it/s]


In [28]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 39839      | 16.78                |
| Number of points covered by less than or equal to 1 image    | 64101      | 27.01                |
| Number of points covered by less than or equal to 3 images   | 98561      | 41.52                |
| Number of points covered by less than or equal to 5 images   | 122569     | 51.64                |


### 5. Estimate memory usage

In [67]:
from ssfm.memory_calculator import memory_calculator

In [68]:
# pointcloud file
npy_file = mesh_file_path
# image file sample
image_file = os.path.join(photos_folder_path, image_list[0])
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(npy_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+-----------------------+
|              Memory Type               |  Memory Required (GB) |
+----------------------------------------+-----------------------+
|      Segmentation for each image       |  0.002336740493774414 |
| Pixel2point association for each image |  0.004673480987548828 |
| Point2pixel association for each image | 0.0003031231462955475 |
|                                        |                       |
|      Segmentation for all images       |   1.4347586631774902  |
| Pixel2point association for all images |   2.8695173263549805  |
| Point2pixel association for all images |  0.18611761182546616  |
|          pc_segmentation_ids           | 0.0015156157314777374 |
|         pc_segmentation_probs          | 0.0015156157314777374 |
|          keyimage_association          |  0.04652940295636654  |
|                 Total                  |   4.539954235777259   |
+----------------------------------------+--------------------

### 6. Run object registration

In [9]:
from ssfm.object_registration import *
from ssfm.post_processing import *

In [10]:
pointcloud_path = os.path.join(scene_dir, 'reconstructions', 'mesh_vertices_color.npy')

keyimage_associations_file_name = 'associations_keyimage.npy'

In [24]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentations_folder_path, associations_folder_path, keyimage_associations_file_name=keyimage_associations_file_name, image_list=image_list)

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)

Processing images: 100%|██████████| 255/255 [02:12<00:00,  1.93it/s]


In [25]:
image_id = 254
semantics_folder_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path, remove_small_N=100, nearest_interpolation=100)

Before removing small semantics: 
maximum of semantics:  5143
number of unique semantics:  686
After removing small semantics: 
number of unique semantics:  88


In [26]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_shuffled.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  87


In [27]:
from validation import *

scene_folder_path = '../../data/scannet/ssfm_valid/scene0011_00'
validator = ScannetValidation(scene_folder_path)
results = validator.validate(np.arange(0.5, 1.0, 0.05))
#results = validator.validate([0.5, 0.6])
mAP = results['AP']
mAR = results['AR']
print('mAP list: ', mAP, ' mAR list: ', mAR)
mAP = np.sum(mAP) / len(mAP)
mAR = np.sum(mAR) / len(mAR)
print('mAP: ', mAP, ' mAR: ', mAR)

Unique semantics in the ground truth:  34
Using the last prediction file:  ../../data/scannet/ssfm_valid/scene0011_00/associations/semantics/semantics_254_shuffled.las
Number of unique semantics in the prediction:  87
Number of unique semantics in the ground truth:  34
TP:  4  FP:  83  FN:  30
TP:  3  FP:  84  FN:  31
TP:  3  FP:  84  FN:  31
TP:  2  FP:  85  FN:  32
TP:  1  FP:  86  FN:  33
TP:  1  FP:  86  FN:  33
TP:  0  FP:  87  FN:  34
TP:  0  FP:  87  FN:  34
TP:  0  FP:  87  FN:  34
TP:  0  FP:  87  FN:  34
mAP list:  [0.04597701149425287, 0.034482758620689655, 0.034482758620689655, 0.022988505747126436, 0.011494252873563218, 0.011494252873563218, 0.0, 0.0, 0.0, 0.0]  mAR list:  [0.11764705882352941, 0.08823529411764706, 0.08823529411764706, 0.058823529411764705, 0.029411764705882353, 0.029411764705882353, 0.0, 0.0, 0.0, 0.0]
mAP:  0.016091954022988506  mAR:  0.041176470588235294
